In [3]:
!rm -rf fever-scorer

In [4]:
!git clone -b release-v2.0 https://github.com/sheffieldnlp/fever-scorer.git

Cloning into 'fever-scorer'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 224 (delta 0), reused 0 (delta 0), pack-reused 219 (from 1)
Receiving objects: 100% (224/224), 1.13 MiB | 9.40 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [5]:
%cd fever-scorer

/content/fever-scorer


In [6]:
!pip install -r requirements.txt


In [7]:
!pip install .


Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Processing /content/fever-scorer
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info
  writing /tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info/SOURCES.txt'
  adding license file 'LICENSE' (matched pattern 'LICEN[CS]E*')
  reading manifest file '/tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  writing manifest file '/tmp/pip-pip-egg-info-dvmtizb9/fever_scorer.egg-info/SOURCES.txt'
  Preparing metada

In [8]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24951 sha256=410254fc43d4bf702d4d898311a62c256d12aee325def93e3ef9c3f5a29474f4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [9]:
import pandas as pd
import nltk  # Make sure NLTK is installed and data downloaded (e.g., nltk.download('punkt'))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
import openai  # For LLM interaction
from openai import OpenAI
import numpy as np
from nltk import Tree, pos_tag, word_tokenize, ne_chunk
from nltk.corpus import stopwords
import numpy as np
from fever.scorer import fever_score # Import the FEVER scorer
from nltk import RegexpParser


# Download the necessary NLTK data files
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
nltk.download('stopwords')
nltk.download('treebank')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_t

True

In [10]:


# Sample instances
instances = [
    {
        "label": "REFUTES",
        "predicted_label": "REFUTES",
        "predicted_evidence": [["Page1", 1]],
        "evidence": [
            [
                [None, None, "Page1", 1],
                [None, None, "Page2", 2],
            ]
        ],
    },
    {
        "label": "SUPPORTS",
        "predicted_label": "SUPPORTS",
        "predicted_evidence": [["Page3", 3]],
        "evidence": [
            [
                [None, None, "Page3", 3]
            ]
        ],
    },
    {
        "label": "NOT ENOUGH INFO",
        "predicted_label": "NOT ENOUGH INFO",
        "predicted_evidence": [],
        "evidence": [],
    },
]

# Calculate scores
strict_score, label_accuracy, precision, recall, f1 = fever_score(instances)

# Display results
print(f"Strict Score: {strict_score:.2f}")
print(f"Label Accuracy: {label_accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Strict Score: 0.67
Label Accuracy: 1.00
Precision: 1.00
Recall: 0.50
F1 Score: 0.67


In [11]:
# Mount google drive
from google.colab import drive
import gc

drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from google.colab import userdata
api_key = userdata.get('openaikey')
client = OpenAI(api_key=api_key)

In [15]:
%cd ../drive/My Drive/SUNY_Poly_DSA598/

/content/drive/My Drive/SUNY_Poly_DSA598


In [16]:
!ls -a

archive			    .git			    presentation
datasets		    .gitignore			    transcribe_voice_notes.ipynb
FEVER_set_creation.ipynb    liar_gpt4omini_base_eval.ipynb  work_documents
FEVER_tuning_archive.ipynb  Module_2_dev.ipynb


In [20]:
%cd ./datasets/FEVER/

/content/drive/MyDrive/SUNY_Poly_DSA598/datasets/FEVER


In [21]:
!ls -a

AVeriTeC   fever2-adversarial.jsonl	    fever-train.jsonl  paper_dev.jsonl	 tabular_sets
.DS_Store  feverous_train_challenges.jsonl  GPT_sets	       paper_test.jsonl  wiki-pages


In [22]:
# Data paths (replace with your actual paths if different)
fever_path = './datasets/FEVER/'
#train_clf_path = f"{fever_path}tabular_clf_paper_dev_train/v1_segmented_n3461_03-29_001.csv"
#valid_clf_path = f"{fever_path}tabular_clf_paper_dev_valid/v1_segmented_n1482_03-29_001.csv"
train_sentEx_path = f"/content/drive/MyDrive/SUNY_Poly_DSA598/datasets/FEVER/tabular_sets/tabular_sentEx_paper_dev_train/v1_segmented_n3461_03-29_001.csv"
valid_sentEx_path = f"/content/drive/MyDrive/SUNY_Poly_DSA598/datasets/FEVER/tabular_sets/tabular_sentEx_paper_dev_valid/v1_segmented_n1482_03-29_001.csv"

# Load datasets
#train_clf = pd.read_csv(train_clf_path)
#valid_clf = pd.read_csv(valid_clf_path)
train_sentEx = pd.read_csv(train_sentEx_path)
valid_sentEx = pd.read_csv(valid_sentEx_path)

In [26]:
# Show the distribution of labels
print(f"Train set label distribution:")
print(train_sentEx['label'].value_counts())
print(f"Valid set label distribution:")
print(valid_sentEx['label'].value_counts())

# Balance the labels by reducing each to the minimum count
min_count = min(train_sentEx['label'].value_counts())
train_sentEx = train_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
min_count = min(valid_sentEx['label'].value_counts())
valid_sentEx = valid_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)

print(f"Train set label distribution after balancing:")
print(train_sentEx['label'].value_counts())
print(f"Valid set label distribution after balancing:")
print(valid_sentEx['label'].value_counts())

Train set label distribution:
label
SUPPORTS           1156
REFUTES            1156
NOT ENOUGH INFO    1149
Name: count, dtype: int64
Valid set label distribution:
label
SUPPORTS           495
REFUTES            495
NOT ENOUGH INFO    488
Name: count, dtype: int64
Train set label distribution after balancing:
label
NOT ENOUGH INFO    1149
REFUTES            1149
SUPPORTS           1149
Name: count, dtype: int64
Valid set label distribution after balancing:
label
NOT ENOUGH INFO    488
REFUTES            488
SUPPORTS           488
Name: count, dtype: int64


<ipython-input-26-03091368d446>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_sentEx = train_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
<ipython-input-26-03091368d446>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valid_sentEx = valid_sentEx.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)


In [27]:



def module_2_semi_supervised_distillation(claim, documents, entities, keywords, verbose=0, debug=False):
    """
    Module 2: Semi-supervised Distillation for sentence extraction.

    Args:
        claim (str): The input claim.
        documents (list of str): List of retrieved documents (full text).
        entity (str):  The main entity in the claim (or None if not found).
        keywords (list of str):  Top keywords from the claim.

    Returns:
        tuple: (list of str, str):  A list of extracted evidence sentences and the exit status ("NOT ENOUGH INFO" or "OK").
    """


    extracted_sentences = []
    max_iterations = min(5, len(documents)) # This will be determined later with actual number of sentences across all docs
    if debug:
      print(f"DEBUG 2.1.1:")
      print(f"\tNumber of documents: {len(documents)}")
      print(f"\tMax iterations: {max_iterations}")
      print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")


    if entities == []:
      if verbose == 1:
        print("No entities found, early exiting...")
        return [], "NOT ENOUGH INFO" # Early exit if no entity

    # convert entities to string
    entities = ", ".join(entities)
    if debug:
      print(f"DEBUG 2.1.2:")
      print(f"\tClaim: {claim}")
      print(f"\tEntities: {entities}")
      print(f"\tKeywords: {keywords}")
      print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")

    for iteration in range(max_iterations):
        # 2.2 Prompt Selection (no agent - programmatic)
        if iteration == 0:
            prompt = f"Retrieve sentences from the list that either support or refute the following claim. Specifically, focus on sentences mentioning {entities}. Order the sentences by relevance, highest first, and return a list separated by the return character. If there are no relevant sentences, respond with 'NOT ENOUGH INFO'. DO NOT CREATE ANY SENTENCES THAT ARE NOT IN THE PROVIDED LIST."
        else:  # Follow up prompts
            prompt = f"You didn’t find enough sentences. Find additional (new) sentences that mention {entities} or other key points in the claim. Order the sentences by relevance, highest first, and return a list separated by the return character. If there are no relevant sentences, respond with 'NOT ENOUGH INFO'. DO NOT CREATE ANY SENTENCES THAT ARE NOT IN THE PROVIDED LIST."


        # 2.3 Sentence Extraction (with pre-filtering)
        #filtered_text = sliding_window_filter(documents, entities, keywords)
        # Simply form the list of document strings into one string joined by \n for now
        filtered_text = "\n".join(documents)
        if debug:
          print(f"DEBUG 2.3.1:")
          print(f"\tFiltered text: {filtered_text}")
          print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")
        new_sentences = extract_sentences_with_llm(claim, filtered_text, prompt, True)

        if new_sentences == ["NOT ENOUGH INFO"]:  # LLM found no relevant sentences

            if iteration == 0:  # No sentences found in the first iteration
                if verbose == 1:
                  print(f"2.3: LLM found no relevant sentences, early exiting...")
                return [], "NOT ENOUGH INFO"  # Early exit if no relevant sentences in first pass.
            else: # No sentences found in subsequent iteration
                if len(extracted_sentences) == 0:
                  if verbose == 1:
                    print(f"2.3: LLM found no relevant sentences in second attempt, early exiting...")
                  return [], "NOT ENOUGH INFO"  # Still not enough info
                break   #Stop iterating and use sentences we found


        # 2.4 Similarity Comparison and Thresholding
        new_sentences = similarity_thresholding(claim, new_sentences, True)

        extracted_sentences.extend(new_sentences)  # Add new sentences to the list
        if debug:
          print(f"DEBUG 2.4.2:")
          print(f"\tAdded {len(new_sentences)} new sentences on iteration {iteration}.")
          print(f"\tTotal extracted sentences: {len(extracted_sentences)}")
          print(f"\tExtracted sentences: {extracted_sentences}")
          print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")
        if len(extracted_sentences) >= min(max_iterations, 5):
            if verbose == 1:
                print(f"2.4: Found enough sentences, exiting...")
            break  # Exit if enough sentences are found.

    if verbose == 1:
      print(f"\tExtracted sentences: {extracted_sentences}")
      print(f"\tNumber of extracted sentences: {len(extracted_sentences)}")
      print(f"\tMax iterations: {max_iterations}")
      print("-------------------------------------------------------------")

    #Check for final early exit
    if len(extracted_sentences) < 5 and max_iterations > 5: #Pilot study will determine best practices here
        return [], "NOT ENOUGH INFO"  # Not enough relevant sentences found after iterations

    return extracted_sentences, "OK"



def sliding_window_filter(documents, entity, keywords, debug=False):
    """
    Performs sliding window filtering based on entity and keywords.

    Args:
        documents (list of str):  List of documents.
        entity (str): The entity to match.
        keywords (list of str): The keywords to match.

    Returns:
        str: Concatenated filtered text.
    """
    # TODO: Implement sliding window filtering logic (using NLTK, spaCy, etc.).
    # This function should concatenate sentences from windows that contain entity/keywords.
    filtered_sentences = []

    for document in documents:
        sentences = nltk.sent_tokenize(document)
        window_size = 3  # Experiment with different window sizes.
        for i in range(len(sentences) - window_size + 1):
            window = sentences[i : i + window_size]
            window_text = " ".join(window)
            if entity in window_text and any(keyword in window_text for keyword in keywords):
                filtered_sentences.extend(window) # Add matching sentences from window to filtered sentences

    filtered_text = "\n".join(filtered_sentences)
    return filtered_text



def extract_sentences_with_llm(claim, filtered_text, prompt, debug=False):
    """
    Extracts sentences using an LLM.

    Args:
        claim (str): The input claim.
        filtered_text (str): The pre-filtered text.
        prompt (str): The prompt for the LLM.

    Returns:
        list of str: Extracted sentences.
    """
    # TODO: Implement LLM interaction (using OpenAI API, etc.).
    # Use the provided prompt and filtered_text to extract sentences with the LLM.

    if debug:
      print(f"DEBUG 2.3.2:")
      print(f"\tPrompt: {prompt}")
      print(f"\tFiltered text: {filtered_text}")
      print("-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts evidence sentences."},
            {"role": "user", "content": f"{prompt}\nClaim: {claim}\nText: {filtered_text}"},
        ],
        max_tokens=512,  # Adjust as needed
        n=1,
        stop=None,  # Or a suitable stop sequence
        temperature=0.9,  # Adjust as needed
    )
    extracted_sentences_raw = response.choices[0].message.content
    extracted_sentences = extracted_sentences_raw.split('\n')  # Assuming sentences are separated by newlines

    return extracted_sentences




def similarity_thresholding(claim, sentences, debug=False):
    """
    Thresholds sentences based on similarity to the claim.

    Args:
        claim (str): The input claim.
        sentences (list of str): The sentences to threshold.

    Returns:
        list of str:  The sentences that meet the similarity threshold.
    """
    # TODO: Implement similarity calculations (ROUGE, TF-IDF, cosine similarity).
    # Return only the sentences that exceed the defined threshold(s).

    threshold = 0.2 # Placeholder, determine empirically.
    filtered_sentences = []

    vectorizer = TfidfVectorizer()  # Initialize TF-IDF vectorizer
    claim_tfidf = vectorizer.fit_transform([claim])
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)


    for sentence in sentences:
        sentence_tfidf = vectorizer.transform([sentence])
        cosine_sim = cosine_similarity(claim_tfidf, sentence_tfidf)[0][0]
        rouge_scores = scorer.score(claim, sentence)
        rouge1_score = rouge_scores['rouge1'].fmeasure  # Example: Use ROUGE-1 F1-score

        if debug:
          print(f"DEBUG 2.4.1:")
          print(f"\tClaim: {claim}")
          print(f"\tSentence: {sentence}")
          print(f"\tCosine Similarity: {cosine_sim}")

        if cosine_sim >= threshold and rouge1_score >= threshold:  # Example: Combine cosine similarity and ROUGE
            filtered_sentences.append(sentence)

    return filtered_sentences


# Entity and keywords extraction function
def extract_entities(text):
    """
    Extracts entities from the text using NLTK's Named Entity Chunker.

    Args:
        text (str): The input text.

    Returns:
        list of str: List of extracted entities.
    """
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tagged_tokens = pos_tag(tokens)
    named_entities = ne_chunk(tagged_tokens)

    entities = []
    for subtree in named_entities:
        if isinstance(subtree, Tree):
            entity = " ".join([word for word, tag in subtree.leaves()])
            entities.append(entity)

    return entities


# Keyword extraction function
def extract_keywords(text):
    """
    Extracts keywords from the text using TF-IDF.

    Args:
        text (str): The input text.

    Returns:
        list of str: List of extracted keywords.
    """
    vectorizer = TfidfVectorizer(stop_words='english', max_features=10)  # Adjust max_features as needed
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    dense = tfidf_matrix.todense()
    denselist = dense.tolist()
    # Convert the list to a NumPy array to use argsort()

    dense_array = np.array(denselist[0])
    keywords = [feature_names[i] for i in dense_array.argsort()[-5:]]  # Get top 5 keywords

    return keywords

# Scorer function for evaluation
def scorer(gold_sentences, extracted_sentences):
    """
    Evaluates the extracted sentences against gold standard sentences.

    Args:
        gold_sentences (list of str): The gold standard sentences.
        extracted_sentences (list of str): The extracted sentences.

    Returns:
        dict: Evaluation scores.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {}

    for gold_sentence in gold_sentences:
        for extracted_sentence in extracted_sentences:
            score = scorer.score(gold_sentence, extracted_sentence)
            scores[extracted_sentence] = score

    # Calculate the macro-F1 score
    total_score = 0
    for sentence, score in scores.items():
        total_score += score['rouge1'].fmeasure
    avg_score = total_score / len(scores) if scores else 0
    return scores, avg_score

# Claim generator function using valid_sentEx
index = 4
def generate_claim(df):
    """
    Generates a claim from the provided DataFrame.
    """
    global index
    if index >= len(df):
        index = 0  # Reset index if it exceeds the DataFrame length
    claim = df.iloc[index]['claim']
    documents = df.iloc[index]['full_text']
    documents = documents.split('\n')
    label = df.iloc[index]['label']
    index += 1
    return claim, documents, label


In [28]:
claim, documents_text, label = generate_claim(valid_sentEx)  # Generate a claim and its associated documents


keywords = extract_keywords(claim)  # Extract keywords from the claim
entities = extract_entities(claim)  # Extract entities from the claim

print(f"DEBUG 1.1:")
print(f"\tClaim: {claim}")
print("Entities:", entities)
print("Keywords:", keywords)


# Convert documents to list of strings if it's a single string
if isinstance(documents_text, str):
    documents = documents_text.split('\n')
elif isinstance(documents_text, list):
  documents = documents_text
else:
  raise Exception(f"documents must be a list of strings, not {type(documents_text)}")

print(f"Claim: {claim}")
print("Entities:", entities)
print("Keywords:", keywords)

extracted_evidence, status = module_2_semi_supervised_distillation(claim, documents, entities, keywords, verbose=1, debug=True)



# Convert extracted evidence to FEVER format
predicted_evidence = []
for sentence in extracted_evidence:
    for doc in documents:
        if sentence in doc:  # or a more robust check if sentences are preprocessed
            line_num = doc.split('\n').index(sentence)  # Assumes sentences are split by newlines in doc
            predicted_evidence.append([f"page_{documents.index(doc)}", line_num])
            break  # Stop searching once sentence is found in a document

# Construct FEVER prediction instance
prediction_instance = {
    "predicted_label": "SUPPORTS" if status == "OK" else "NOT ENOUGH INFO",  # Or REFUTES based on your logic
    "predicted_evidence": predicted_evidence,

}


# Get evidence for the claim (adapt this based on your gold standard data format)
actual_evidence = []
gold_sentences = valid_sentEx[valid_sentEx["claim"] == claim]["evidence_sentences"].iloc[0].split('\n')
for sentence in gold_sentences:
    for doc in documents:
        if sentence in doc:
            line_num = doc.split('\n').index(sentence)
            actual_evidence.append([f"page_{documents.index(doc)}", line_num])
            break

# Construct the actual evidence (gold standard) dictionary:

actual_instance = {"label": label, "evidence": [actual_evidence]}

# Calculate FEVER score
strict_score, label_accuracy, precision, recall, f1 = fever_score([prediction_instance], [actual_instance])
fever_results = {
    'strict_score': strict_score,
    'label_accuracy': label_accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

print("FEVER Scores:", fever_results)
print("-------------------------------------------------------------")


if status == "OK":
    print("Extracted Evidence Sentences:")
    for sentence in extracted_evidence:
        print('\t' + sentence)
else:
    print(f"Claim classification: {status}")




# Example evaluation using your existing scorer (optional, but good for comparison)
print("\nAdditional Evaluation Scores (ROUGE, etc.):")
scores, avg_score = scorer(gold_sentences, set(extracted_evidence)) # changed documents to gold_sentences for ROUGE evaluation, make sure to set to correct variable, most likely will be called evidence_sentences
for sentence, score in scores.items():
    print(f"Sentence: {sentence}")
    print(f"Scores: {score}")
    print("-------------------------------------------------------------")
print(f"Average Score: {avg_score}")

DEBUG 1.1:
	Claim: Goosebumps (film) was directed by a Baptist.
Entities: ['Goosebumps', 'Baptist']
Keywords: ['baptist', 'directed', 'film', 'goosebumps']
Claim: Goosebumps (film) was directed by a Baptist.
Entities: ['Goosebumps', 'Baptist']
Keywords: ['baptist', 'directed', 'film', 'goosebumps']
DEBUG 2.1.1:
	Number of documents: 1
	Max iterations: 1
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
DEBUG 2.1.2:
	Claim: Goosebumps (film) was directed by a Baptist.
	Entities: Goosebumps, Baptist
	Keywords: ['baptist', 'directed', 'film', 'goosebumps']
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
DEBUG 2.3.1:
	Filtered text: The Tennis South Invitational, was a men's tennis tournament founded in 1 as the Mississippi International Indoor Tennis Championships. It was played at the Mississippi Coliseum in Jackson, Mississippi in the United States until 1977. The event was played as part of the USLTA Indoor Circuit from 1 through 19751 and became a World Championship Tennis event in 1976. In its

ValueError: 'Ken Rosewall was the only multiple singles champion, winning the title in 1 and 1976.' is not in list

In [ ]:



extracted_evidence, status = module_2_semi_supervised_distillation(claim, documents, entities, keywords, verbose=1, debug=True)

if status == "OK":
    print("Extracted Evidence Sentences:")
    for sentence in extracted_evidence:
        print('\t' + sentence)
    print(f"Claim classification: {status}")
    print(f"Label: {label}")
else:
    print(f"Claim classification: {status}")
    print(f"Label: {label}")

# Evaluation
# Assuming you have a gold standard set of sentences for evaluation

print("Evaluation Scores:")
scores, avg_score = scorer(documents, set(extracted_evidence))
for sentence, score in scores.items():
    print(f"Sentence: {sentence}")
    print(f"Scores: {score}")
    print("-------------------------------------------------------------")
print(f"Average Score: {avg_score}")  # Adjust based on your scoring logic